In [1]:
import pandas as pd
import numpy as np
import torch


# from run_no_val import run

from run_with_val import run

from model_3d.Comenet_NMR import ComENet

from load_Glycosciencedb_3d import create_graph_experiment

from load_GODDESS_3d import GODDESSDataset, split_test_val
from tqdm import tqdm
from run_fine_tune import run_tune
np.random.seed(9721)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create = create_graph_experiment()

train_data_exp, test_data_exp = Create.create_all_graph_list()

# number of training set 1

num_train1 = int(len(train_data_exp) / 0.8 * 0.6)

num_train2 = int(len(train_data_exp) - num_train1)

indices_train1 = np.random.choice(len(train_data_exp), num_train1, replace=False)

indices_train2 = np.setdiff1d(np.array(range(0, len(train_data_exp))), indices_train1)

train_data_1 = [train_data_exp[i] for i in indices_train1]
train_data_2 = [train_data_exp[i] for i in indices_train2]

num_val = int(len(test_data_exp) / 2)
num_test = int(len(test_data_exp) / 2)

indices_val = np.random.choice(len(test_data_exp), num_val, replace=False)

indices_test = np.setdiff1d(np.array(range(0, len(test_data_exp))), indices_val)

val_data = [test_data_exp[i] for i in indices_val]
test_data = [test_data_exp[i] for i in indices_test]

loss_func = torch.nn.L1Loss()
run3d_tune = run_tune()

node_embedding_size = train_data_exp[0].z.shape[1]

--------------------------loading NMR Graph List-------------------------------


100%|█████████████████████████████████████████| 299/299 [00:07<00:00, 41.81it/s]


In [3]:
batch_size = [2, 4, 16]
hidden_channels = [32, 64, 128, 256]
lr = [0.001, 0.01]
num_layer = [2, 3]
c_list = [4.0, 5.0]
best_param = {}
best_param['b'] = 0
best_param['h'] = 0
best_param['c'] = 0
best_param['l'] = 0
best_param['n'] = 0

best_rmse = 10000

In [4]:
# # for b in tqdm(batch_size):
    
# for h in hidden_channels:
        
# # for l in lr:
                
# # for n in tqdm(num_layer):

# #for c in c_list:
#     b = 2
# #     h = 256
#     c = 4.0
#     n = 2
#     l = 0.001
    
#     model = ComENet(cutoff = c, hidden_channels=h, middle_channels=128, num_layers=n, num_output_layers=1, in_embed_size = node_embedding_size)

#     train_loss_list, test_rmse_list = run3d_tune.run(device=device, train_dataset = train_data_1, test_dataset = train_data_2,
#                             model = model, loss_func=loss_func,
#                             epochs=100, batch_size=b, vt_batch_size= 2, lr=l, lr_decay_factor=0.5, lr_decay_step_size=15)

#     indx = np.where(np.array(train_loss_list) == min(train_loss_list))[0][0]
#     current_rmse = test_rmse_list[indx]


#     if current_rmse < best_rmse:
#         best_rmse = current_rmse
#         best_param['b'] = b
#         best_param['h'] = h
#         best_param['c'] = c
#         best_param['l'] = l
#         best_param['n'] = n
#     del model

In [5]:
b = 4
h = 256
c = 4.0
n = 2
l = 0.001

In [6]:
model = ComENet(cutoff = c, hidden_channels=h, middle_channels=128, num_layers=n, num_output_layers=1, in_embed_size = node_embedding_size)

In [7]:
loss_func = torch.nn.L1Loss()
run3d_final= run()


import time
start_time = time.time()
print(start_time)
train_loss_list, test_rmse_list = run3d_final.run(device=device, train_dataset = train_data_exp, valid_dataset = val_data, 
                                                  test_dataset = test_data,
                                            model = model, loss_func=loss_func,
                                            epochs=200, batch_size=b, vt_batch_size= 2, lr=l, lr_decay_factor=0.5, lr_decay_step_size=15,
                                            )

print("--- %s seconds ---" % (time.time() - start_time))

1695947596.9370477
#Params: 1780481

=====Epoch 1

Training...


100%|███████████████████████████████████████████| 60/60 [00:01<00:00, 33.89it/s]



Evaluating...



100%|██████████████████████████████████████████| 15/15 [00:00<00:00, 120.31it/s]

use_tensor


Testing...


...



Evaluating...



100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 28.56it/s]

use_tensor


Testing...



100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 33.08it/s]

use_tensor

{'Train': 0.009551134706028582, 'Valid': 0.13737187, 'Test': 0.15835333}
Best validation C RMSE so far: 0.1346568912267685
Test C MAE when got best validation result: 0.15765933692455292
--- 712.2998955249786 seconds ---


In [8]:
251.44 / 200

1.2572